In [ ]:
%load_ext autoreload
%autoreload 2
from allennlp.commands.evaluate import *
from kb.include_all import *
from allennlp.nn import util as nn_util
from allennlp.common.tqdm import Tqdm
import torch
import warnings
warnings.filterwarnings("ignore")
archive_file = "knowbert_wiki_wordnet_model"
cuda_device = -1


In [ ]:
import logging

logger = logging.getLogger()  # 不加名称设置root logger
logger.setLevel(logging.INFO)
formatter = logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s: - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S')

# 使用FileHandler输出到文件
fh = logging.FileHandler('log.txt')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)

# 使用StreamHandler输出到屏幕
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

# 添加两个Handler
logger.addHandler(ch)
logger.addHandler(fh)

In [ ]:
archive = load_archive(archive_file, cuda_device)
config = archive.config
prepare_environment(config)

In [ ]:
config = Params.from_file("/home/scruple/kb/knowbert_wiki_wordnet_model/config.json")

In [ ]:
reader_params = config.pop('dataset_reader')
# print(reader_params.as_dict())
if reader_params['type'] == 'multitask_reader':
    reader_params = reader_params['dataset_readers']['language_modeling']
# reader_params['num_workers'] = 0
validation_reader_params = {
    "type": "food",
    "tokenizer_and_candidate_generator": reader_params['base_reader']['tokenizer_and_candidate_generator'].as_dict()
}
dataset_reader = DatasetReader.from_params(Params(validation_reader_params))
vocab = dataset_reader._tokenizer_and_candidate_generator.bert_tokenizer.vocab
token2word = {}
for k, v in vocab.items():
    token2word[v] = k

In [ ]:
instances = dataset_reader.read("../data/food_compact.tsv")

In [ ]:
print(instances[69]["tokens"])

In [ ]:
model = archive.model
model.eval()


print("start")
# metrics = evaluate(model, instances, iterator, cuda_device, "")
data_iterator = DataIterator.from_params(Params(
    {"type": "basic", "batch_size": 1}
))
data_iterator.index_with(model.vocab)
iterator = data_iterator(instances,
                            num_epochs=1,
                            shuffle=False)
logger.info("Iterating over dataset")
generator_tqdm = Tqdm.tqdm(iterator, total=data_iterator.get_num_batches(instances))
with open('prompt2.txt', 'wt') as f:
    for instance in generator_tqdm:
        batch = nn_util.move_to_device(instance, cuda_device)
        output_dict = model(**batch)
        pooled_output = output_dict.get("pooled_output")
        contextual_embeddings = output_dict.get("contextual_embeddings")
        prediction_scores, seq_relationship_score = model.pretraining_heads(
        contextual_embeddings, pooled_output
        )
        prediction_scores = prediction_scores.view(-1, prediction_scores.shape[-1])
        # print("================")
        # print(prediction_scores)
        # print("================")
        idx = torch.argmax(prediction_scores, dim = 1)
        line = ""
        for i in idx.cpu().numpy():
            line += token2word[i] + " "
        
        print(line, file=f)

    